# GuideLLM Benchmark 🧪

We want a quick way to **benchmark** an LLM endpoint before putting anything in production.  
For that we can use the **`guidellm benchmark`** CLI 🙌

We'll keep it simple and **emulate tokens** to measure basic performance.  
You can tweak parameters later to suit your stack.

## Set-up

Make sure `guidellm` is available in this environment.  
If not, use the cell below.

In [ ]:
!pip install -q guidellm

## Benchmark target 🎯

Point to your endpoint and model. Keep names consistent with your serving stack.

In [ ]:
# ---- Target & Model (edit these) ----
target = "https://llama32-ai501.apps.cluster-tcdr7.tcdr7.sandbox1789.opentlc.com/v1"  # endpoint URL
model = "llama32"  # logical model name on the target
processor = "RedHatAI/Llama-3.2-3B-Instruct-quantized.w8a8"  # serving identifier

## Emulated data 📦

We'll emulate prompts/outputs with fixed token sizes.  
Great for **smoke tests** and **baseline throughput**.

In [ ]:
# ---- Workload (edit this) ----
data = {
    "type": "emulated",
    "prompt_tokens": 512,
    "output_tokens": 128,
}
rate_type = "synchronous"   # pacing type
max_seconds = 60            # total duration (seconds)
output_path = "benchmark-results.yaml"  # where results are written

# Optional passthrough flags, e.g.:
# extra_flags = "--concurrency=4 --headers-file=/tmp/headers.json"
extra_flags = ""

## Preview the command 👀

Guidellm is ran as a CLI rather than a Python library, so here we compile the command before we run it.  
This let's us check the exact CLI before running.

In [ ]:
import json, shlex

def build_guidellm_command(
    target: str,
    model: str,
    processor: str,
    data: dict,
    output_path: str,
    rate_type: str,
    max_seconds: int,
    extra_flags: str = "",
):
    data_json = json.dumps(data, separators=(",", ":"))
    cmd_list = [
        "guidellm", "benchmark",
        f"--target={target}",
        f"--model={model}",
        f"--processor={processor}",
        f"--data={data_json}",
        f"--output-path={output_path}",
        f"--rate-type={rate_type}",
        f"--max-seconds={int(max_seconds)}",
    ]
    if extra_flags.strip():
        cmd_list.extend(extra_flags.strip().split())

    # pretty shell string (quotes + JSON wrapped in single quotes)
    data_shell = f"--data='{data_json}'"
    parts = [
        "guidellm benchmark",
        f"--target={shlex.quote(target)}",
        f"--model={shlex.quote(model)}",
        f"--processor={shlex.quote(processor)}",
        data_shell,
        f"--output-path={shlex.quote(output_path)}",
        f"--rate-type={shlex.quote(rate_type)}",
        f"--max-seconds={int(max_seconds)}",
    ]
    if extra_flags.strip():
        parts.append(extra_flags)

    cmd_shell = " ".join(parts)
    return cmd_list, cmd_shell

cmd_list, cmd_shell = build_guidellm_command(
    target, model, processor, data, output_path, rate_type, max_seconds, extra_flags
)

print("Shell command:\n")
print(cmd_shell)
print("\nSubprocess list:\n")
print(cmd_list)

## Run it ▶️

Execute the benchmark and stream logs here.

In [ ]:
import shutil, subprocess, sys

if shutil.which("guidellm") is None:
    raise RuntimeError(
        "The 'guidellm' CLI is not available in this environment. "
        "Install it in the Set-up section and re-run."
    )

print("Starting benchmark...\n")
process = subprocess.Popen(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
for line in process.stdout:
    sys.stdout.write(line)
ret = process.wait()

print(f"\nDone. Exit code: {ret}")
if ret != 0:
    print("Non-zero exit code. Review the logs above.")

## Inspect results 📝

Quick peek at the output file to confirm success.

In [ ]:
from pathlib import Path

p = Path(output_path)
if p.exists():
    print(f"Results: {p.resolve()}")
    try:
        print("\n--- First 200 lines ---\n")
        with p.open("r", encoding="utf-8", errors="replace") as f:
            for i, line in enumerate(f):
                if i >= 200:
                    print("... (truncated)")
                    break
                print(line.rstrip("\n"))
    except Exception as e:
        print(f"Preview error: {e}")
else:
    print(f"Not found: {p}. Double-check --output-path.")

## Tips & tweaks 💡

Feel free to play around with it some more, here are things you can try:

- Increase throughput: `--concurrency=4`  
- Limit requests instead of seconds: `--max-requests=N`  
- Change format (if supported): `--output-format=yaml|json`  